In [2]:
import Pkg; Pkg.activate("..")

  Activating environment at `~/Projects/comp441/latentplan.jl/Project.toml`


In [7]:
include("datasets/sequence.jl")
include("datasets/d4rl.jl")

Main.D4RL

In [9]:
using .Sequence: SequenceDataset
using .D4RL: load_environment

In [10]:
env = load_environment("halfcheetah-medium-expert-v2")

/home/erciyes/miniconda3/envs/conda_jl/lib/python3.10/site-packages/gym/envs/mujoco/mujoco_env.py:237: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/home/erciyes/miniconda3/envs/conda_jl/lib/python3.10/site-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/erciyes/miniconda3/envs/conda_jl/lib/python3.10/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/erciyes/mini

PyObject <d4rl.utils.wrappers.NormalizedBoxEnv object at 0x2b0f6f9e1ba0>

In [11]:
dataset = env.get_dataset()

load datafile: 100%|█████████████████████████████| 9/9 [00:04<00:00,  1.90it/s]


Dict{Any, Any} with 9 entries:
  "observations"           => Float32[0.0198319 -0.0895013 … 0.0684249 -0.13811…
  "infos/qpos"             => [0.0851197 0.0198319 … 0.0603905 -0.0875389; 0.08…
  "actions"                => Float32[-0.222937 -0.735948 … -0.841655 0.43432; …
  "terminals"              => Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0,…
  "timeouts"               => Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0,…
  "next_observations"      => Float32[-0.00384861 -0.0523943 … -9.28856 4.73286…
  "rewards"                => Float32[-0.20008, -1.19815, -0.690752, 0.268202, …
  "infos/qvel"             => [-0.0187761 -0.167599 … 0.0684249 -0.138116; -0.0…
  "infos/action_log_probs" => Float32[0.0623677, 10.6938, 9.81105, 6.45672, 5.5…

In [15]:
done_bool = dataset["terminals"][1]
final_timestep = dataset["timeouts"][1]

false

In [17]:
done_bool += final_timestep

0

In [19]:
using Statistics: mean, std

In [20]:
observations = dataset["observations"]

2000000×17 Matrix{Float32}:
  0.0198319   -0.0895013   …    0.113651    0.0684249   -0.138116
 -0.00384861  -0.0523943        4.50684    -9.28856      4.73286
 -0.0552984   -7.78502f-5      -7.08117    -1.40371      7.5524
 -0.129968     0.00229594      -7.01441     2.69173     -1.6729
 -0.132119     0.128255        -3.00539     0.178223   -15.0526
 -0.126367     0.161999    …   10.4278     10.51        -0.291503
 -0.129755     0.0882672       12.4906     -2.46865      2.44935
 -0.155284    -0.00775645       5.47621     0.355577    14.5446
 -0.153599    -0.0901696       -9.18679     1.47699      0.455037
 -0.0975235   -0.0550852      -16.2344      2.44634     -1.38807
 -0.0559308    0.018281    …   -6.32778     1.75775    -12.9951
 -0.0393649   -0.0118448       10.8271     -4.48932      0.83587
 -0.0377374   -0.00609548       6.15782     2.5811      11.7766
  ⋮                        ⋱                ⋮          
 -0.0957317    0.221304         0.0232196  -6.80618      4.34442
 -0.09378

In [23]:
mean(observations, dims=1)

1×17 Matrix{Float32}:
 -0.0566751  0.0243704  -0.0616051  …  0.587946  -0.240905  -0.0301841